# Jetracer Data Labeling Tool
Author: George Gorospe, george.e.gorospe@gmail.com
MIT License
Copyright (c) 2021-2023 George Gorospe

About: This tool is used to add a label to unlabeled data collected on the jetracer platform. This is done by renaming the data file, an image, with the x, y, coordinates of the desired path/track/trajectory. 

Required: ipywidgets library & clickable widgets library.
The libraries are used to present the data and collect the new label for the data.

In [9]:
# IPython Libraries for display and widgets
import traitlets
import ipywidgets.widgets as widgets
from IPython.display import display
from jupyter_clickable_image_widget import ClickableImageWidget

# We'll use the bgr8_to_jpeg transform to help us in displaying the images
from jetcam.utils import bgr8_to_jpeg


# Python basic pakcages for image annotation
from uuid import uuid1
import os
import json
import glob
import datetime
import numpy as np
import cv2
import time

# Useful Variables 
index = 0
img = ''
img_path = ''
last_labeled_image_path = ''

## Selecting the Unlabeled Data
Next we'll select the unlabled data. This data will not be modified, instead we'll create a new folder and placed the labeled data in the new folder.

The new folder will have same name and location as your selected data set but "LABELED" will be appended to the end of the folder name. 

In [10]:
# Loading data to be labeled

# ************ Select the Directory with your Dataset ***********
DATASET_DIR = "./data/office_data"

# You can use these print statement to ensure that you've successfully located your data
#print(os.getcwd()) # This will show you your current working directory
#print(os.listdir(DATASET_DIR)) # You should see a list of the files in your directory.

# Creating a new directory for labeled data based on the previous directory
# we have this "try/except" statement because these next functions can throw an error if the directories exist already
try:
    LABELED_DATASET_DIR = DATASET_DIR +'_LABELED' 
    os.makedirs(LABELED_DATASET_DIR)
    print('Created new folder: ' + LABELED_DATASET_DIR)
except FileExistsError:
    print('Directories not created becasue they already exist')
    index = len(glob.glob(os.path.join(DATASET_DIR+'_LABELED','*.jpg')))



# Information about the dataset, number of data points and a listing of the data points.
file_number =  len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
file_list = os.listdir(DATASET_DIR)



# Creating blank image that can be used when there is no image to display
blank = np.ones((224,224,3))
blank = bgr8_to_jpeg(blank)

Directories not created becasue they already exist


Next we'll do some setup for the user interface that will help us label the data.

Again, we'll use widgets to both show examples how to label data and actual data to be labeled.

Also, we'll reference some labeled data as examples of how to properly label data.

In [11]:
# Widget Setup

# Creating a clickable image widget, labeling_widget, we can use it to label the data
# Also creating a labeled_data_widget to visualize the label we've just applied to the data
labeling_widget = ClickableImageWidget(width=224,height=224)
labeled_data_widget = widgets.Image(width=224,height=224)

# A callback for the count textbox widget
#TODO adjust label width
label_layout = widgets.Layout(width='224px', height='30px')
count_widget = widgets.IntText(
    description='Dataset: ', 
    value=len(glob.glob(os.path.join(DATASET_DIR, '*.jpg'))), 
    layout=label_layout,)
labeled_count_widget = widgets.IntText(
    description='Labeled Set: ',
    value=len(glob.glob(os.path.join(LABELED_DATASET_DIR, '*.jpg'))), 
    layout=label_layout,)

# Delete data point widget
button_layout = widgets.Layout(width='224px', height='60px') # Width was 148px

delete_button = widgets.Button(
    description = 'Delete Bad Data Point',
    disabled = False,
    button_style = 'danger',
    layout=button_layout,
    )

mislabeled_data = widgets.Button(
    description = 'Delete Mislabeled Data',
    disabled = False,
    button_style = 'danger',
    layout=button_layout,

)

# Creating an array of images to serve as examples for labeling data.
img1 = widgets.Image(width=224,height=224); img1.value = open("./labeled_data_examples/ex_1.png",'rb').read()
img2 = widgets.Image(width=224,height=224); img2.value = open("./labeled_data_examples/ex_2.png",'rb').read()
img3 = widgets.Image(width=224,height=224); img3.value = open("./labeled_data_examples/ex_3.png",'rb').read()
img4 = widgets.Image(width=224,height=224); img4.value = open("./labeled_data_examples/ex_4.png",'rb').read()
img5 = widgets.Image(width=224,height=224); img5.value = open("./labeled_data_examples/ex_5.png",'rb').read()

example_text = widgets.HTML(value="The following images are examples of properly labeled data. Consider where you would like the car to go next. If there is a corner, aim for a sub-point in the corner rather than the end of the corner.")
examples = widgets.HBox([img1, img2, img3, img4, img5])
instruction_text = widgets.HTML(value="Click within the image to add a label.")

## The initial value of the labeling widget and the saved_snapshot_widget

# Load the first image and dispay it with the snapshot widget
index = len(glob.glob(os.path.join(LABELED_DATASET_DIR, '*.jpg'))) + 1 # Index for file from file_list
img_path = os.path.join(DATASET_DIR,file_list[index])
img = open(img_path,'rb').read()
labeling_widget.value = img

# At the start, there is no data to display on the labeled_data widget, so it is blank
labeled_data_widget.value = blank


## Call-back function handles
Next we'll setup the call-back function handles. These are functions that are called when events occur on the user interface.

There are two callback functions, label_data and delete_data_point. These powerful functions can create new files and delete files from the file system using the 'os' library. 

Each callback function must also be connected to their respective clickable_widget or button.

In [12]:
# Setup of the label_data function
# About: the label_data function is called when you click on the labeling_widget. 
#The x and y coordinates of the location you clicked, are passed to the label_data function.

# Function handel, this function is called each time we click on the labeling widget
def label_data(_, content, msg):
    global index, img, img_path, last_labeled_image_path, labeling_widget, labeled_data_widget
    if content['event'] == 'click':
        # Take x & y coordinates from the click event
        data = content['eventData']
        x = data['offsetX']
        y = data['offsetY']
        
        # Create a new filename with the coordinates appended, then save the file in the new directory.
        uuid = 'xy_%03d_%03d_%s' % (x, y, uuid1()) #uuid(1) creates a unique string each time it's called
        image_path = os.path.join(LABELED_DATASET_DIR, uuid + '.jpg') # Building the path & filename for the new labeled data point
        last_labeled_image_path = image_path # store new name for labeled data, we may need to use to delete badly labeled data
        #labeled_data_widget.value
        with open(image_path, 'wb') as f:
            f.write(img)
            
        # display data with our new label (shown as a green circle), image is read with CV2, resulting in a numpy array that can be edited.
        labeled_data = cv2.imread(image_path)
        labeled_data = cv2.circle(labeled_data, (x, y), 8, (0, 255, 0), 3)
        labeled_data = bgr8_to_jpeg(labeled_data)
        labeled_data_widget.value = labeled_data
        
        # Display the next image for labeling
        # Increment the index and update labeling widget image
        index = index + 1
        img_path = os.path.join(DATASET_DIR,file_list[index])
        img = open(img_path,'rb').read()
        labeling_widget.value = img
        
        
        # Update user interfacex
        labeled_count_widget.value = len(glob.glob(os.path.join(LABELED_DATASET_DIR, '*.jpg')))
        display(data_labeling_interface)

In [13]:
# Setup of the delete_data_point function
# About: this function is called when the delete_datapoint button is pressed
#        the function deletes from the original dataset the selected data poinnt

def delete_data_point():
    global img, index, file_list, file_number, labeling_widget, labeled_data_widget
    current_image_path = os.path.join(DATASET_DIR,file_list[index-1])
    
    # Delete the current image
    os.remove(current_image_path)
    
    # Update file_list and file_number
    file_number =  len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
    file_list = os.listdir(DATASET_DIR)
    
    # Update the user interface with a new image and update counters
    img_path = os.path.join(DATASET_DIR,file_list[index])
    img = open(img_path,'rb').read()
    labeling_widget.value = img
    
    count_widget.value = len(glob.glob(os.path.join(DATASET_DIR, '*.jpg')))
    display(data_labeling_interface)
    
def delete_mislabeled_data_point():
    #global last_labeled_image_path
    os.remove(last_labeled_image_path)

    # Update User Interface
    labeled_data_widget.value = blank

    labeled_count_widget.value = len(glob.glob(os.path.join(LABELED_DATASET_DIR, '*.jpg')))
    display(data_labeling_interface)


## User Interface setup
Jupyter labs notebooks are capable of creating very useful interfaces with displays and buttons.

Below we'll create a somewhat complex interface, using VBoxes and HBoxes. Additionally, we'll also show examples of properly labed data.

In [14]:
# User interface Setup
# connecting the labeling_widget and the save_snapshot function
labeling_widget.on_msg(label_data)
# Connecting the delete_button to the delete_data_point function
delete_button.on_click(lambda x: delete_data_point())
# Connecting the mislabeled_data button to the delete_mislabeled_data_point function
mislabeled_data.on_click(lambda x: delete_mislabeled_data_point())

# The user interface configuration seen below is built from VBoxes (verticlly arranged items), and HBoxes (horizontally arranged items).
image_display = widgets.HBox([labeling_widget, labeled_data_widget])
button_display = widgets.HBox([delete_button, mislabeled_data])
counters_display = widgets.HBox([count_widget, labeled_count_widget])
data_labeling_interface = widgets.VBox([image_display, button_display, counters_display])

# Now that we've created the widgets and attached callback functions to each of them
# we can display the widgets and create the user interface for labeling our data
#display(example_text)
#display(examples)
display(instruction_text)
display(data_labeling_interface)

HTML(value='Click within the image to add a label.')